In [2]:
from PIL import Image
import numpy as np
import os

MESSAGE_DELIMITER = "####END_MESSAGE####"

def text_to_binary(text):
    """Converts a string of text into a binary string."""
    # Convert each character to its ASCII value, then to an 8-bit binary string.
    binary_string = ''.join(format(ord(char), '08b') for char in text)
    return binary_string

def binary_to_text(binary_string):
    """Converts a binary string back into a text string."""
    text_message = ""
    #Iterate through the binary string in chunks of 8 bits (one byte).
    for i in range(0, len(binary_string), 8):
        byte = binary_string[i:i+8]
        # Convert the 8-bit binary string to an integer, then to a character.
        text_message += chr(int(byte, 2))
    return text_message

#Steganography Functions

def encode_image(image_path, secret_message, output_path="encoded_image.png"):
    """
    Hides a secret message within an image using the LSB method.

    Args:
        image_path (str): The path to the cover image.
        secret_message (str): The message to hide.
        output_path (str): The path to save the encoded image.
    """
    try:
        img = Image.open(image_path)
    except FileNotFoundError:
        print(f"Error: Image file not found at {image_path}")
        return
    except Exception as e:
        print(f"Error opening image: {e}")
        return

    # Ensure the image is in RGB format for consistent channel access.
    # If the image is 'L' (grayscale) or 'RGBA' (with alpha), convert to 'RGB'.
    if img.mode != 'RGB':
        img = img.convert('RGB')

    full_message_binary = text_to_binary(secret_message + MESSAGE_DELIMITER)
    message_length = len(full_message_binary)

    width, height = img.size
    max_capacity_bits = width * height * 3

    if message_length > max_capacity_bits:
        print(f"Error: Message is too long for the image.")
        print(f"Message bits: {message_length}, Image capacity bits: {max_capacity_bits}")
        return

    # Create a mutable copy of the image pixels to modify.
    # img.getdata() returns a sequence of pixel values.
    pixels = list(img.getdata())
    encoded_pixels = []
    message_index = 0

    print(f"Embedding message ({message_length} bits) into the image...")

    for pixel in pixels:
        r, g, b = pixel
        new_r, new_g, new_b = r, g, b # Start with original pixel values

        # Check if there are more message bits to embed
        if message_index < message_length:
            # Modify the LSB of the red channel
            # pixel & 0xFE clears the LSB (e.g., 10101011 -> 10101010)
            # | bit adds the message bit to the LSB (e.g., 10101010 | 1 -> 10101011)
            new_r = (r & 0xFE) | int(full_message_binary[message_index])
            message_index += 1

        if message_index < message_length:
            # Modify the LSB of the green channel
            new_g = (g & 0xFE) | int(full_message_binary[message_index])
            message_index += 1

        if message_index < message_length:
            # Modify the LSB of the blue channel
            new_b = (b & 0xFE) | int(full_message_binary[message_index])
            message_index += 1

        encoded_pixels.append((new_r, new_g, new_b))

        # If all message bits are embedded, copy the remaining original pixels
        if message_index >= message_length:
            encoded_pixels.extend(pixels[len(encoded_pixels):])
            break # Exit the loop as message is fully embedded

    # Create a new image from the modified pixels
    encoded_img = Image.new(img.mode, img.size)
    encoded_img.putdata(encoded_pixels)

    # Save the encoded image
    encoded_img.save(output_path)
    print(f"Message successfully hidden! Encoded image saved as '{output_path}'")

def decode_image(encoded_image_path):
    """
    Extracts a secret message from an image encoded with the LSB method.

    Args:
        encoded_image_path (str): The path to the encoded image.

    Returns:
        str: The extracted secret message, or None if an error occurs.
    """
    try:
        img = Image.open(encoded_image_path)
    except FileNotFoundError:
        print(f"Error: Encoded image file not found at {encoded_image_path}")
        return None
    except Exception as e:
        print(f"Error opening image: {e}")
        return None

    if img.mode != 'RGB':
        img = img.convert('RGB')

    pixels = list(img.getdata())
    extracted_bits = []

    print(f"Extracting message from '{encoded_image_path}'...")

    for pixel in pixels:
        r, g, b = pixel
        # Extract the LSB of each color
        extracted_bits.append(str(r & 1)) # LSB of red
        extracted_bits.append(str(g & 1)) # LSB of green
        extracted_bits.append(str(b & 1)) # LSB of blue

    binary_message = "".join(extracted_bits)

    # We need to find the delimiter to know where the actual message ends.
    current_message = ""
    # The delimiter itself is 8 bits per character.
    delimiter_length_bits = len(MESSAGE_DELIMITER) * 8

    for i in range(0, len(binary_message), 8):
        byte = binary_message[i:i+8]
        if not byte: # Handle cases where we run out of bits
            break
        char = chr(int(byte, 2))
        current_message += char

        # Check if the delimiter is present at the end of the current_message
        if current_message.endswith(MESSAGE_DELIMITER):
            # If found, return the message portion before the delimiter
            return current_message[:-len(MESSAGE_DELIMITER)]

    print("Delimiter not found or message incomplete. Returning all extracted data.")
    # If the delimiter is not found, it means the message was either not fully embedded
    # or the image was not encoded with this method.
    return current_message # Return whatever was extracted

# --- Example Usage ---

if __name__ == "__main__":
    # 1. Prepare your image
    # Provide the name and path of your image file here.
    # Example: input_image_name = "my_picture.png"
    # Ensure this image file is in the same folder as your Jupyter/Colab notebook.
    input_image_name = input_image_name = input_image_name = input_image_name = "image1.jpg" # Now it's just the filename because it's in Colab's root
    if not os.path.exists(input_image_name):
        print(f"Error: Your input image '{input_image_name}' was not found.")
        print("Please ensure the image file is in the same folder as your notebook.")
        # Exit the program if the image is not found.
        exit() # In Jupyter/Colab, you can also use sys.exit() if this doesn't work as expected

    # 2. Define the secret message
    secret_message_to_hide = "Hello! This is a top secret message hidden using LSB steganography. This is a test to see how well it works!"

    # 3. Encode the message into the image
    encoded_output_image_name = "lsb_encoded_image.png"
    encode_image(input_image_name, secret_message_to_hide, encoded_output_image_name)

    print("\n--- Decoding Process ---")

    # 4. Decode the message from the encoded image
    extracted_message = decode_image(encoded_output_image_name)

    if extracted_message is not None:
        print(f"\nExtracted message: '{extracted_message}'")

        # 5. Verify if the extracted message matches the original
        if extracted_message == secret_message_to_hide:
            print("\nVerification: The extracted message matches the original! Steganography successful.")
        else:
            print("\nVerification: Mismatch between original and extracted message. Something went wrong.")
    else:
        print("\nDecoding failed or no message was extracted.")

Embedding message (1016 bits) into the image...
Message successfully hidden! Encoded image saved as 'lsb_encoded_image.png'

--- Decoding Process ---
Extracting message from 'lsb_encoded_image.png'...

Extracted message: 'Hello! This is a top secret message hidden using LSB steganography. This is a test to see how well it works!'

Verification: The extracted message matches the original! Steganography successful.
